# Integration Tests

In [3]:
import requests
from pandas import DataFrame

In [5]:
url = "http://127.0.0.1:8000/monsters"

## Read All Monsters

In [18]:
df = DataFrame(requests.get(url).json())
original_shape = df.shape
df

,name,type,level,health
0,Orc Captain,Giant,33,170
1,Mummy,Undead,38,209
2,Fire Mephit,Elemental,33,169
3,Cave Troll,Giant,40,191
4,Black Dragon,Dragon,75,436
...,...,...,...,...
995,Earth Mephit,Elemental,32,179
996,Troll,Giant,39,206
997,Ghost,Undead,54,311
998,Black Dragon Welp,Dragon,44,235


### Create Monster

In [8]:
requests.post(url, json=[
    {
        "name": "test monster 1",
        "type": "test",
        "level": 0,
        "health": 0,
    },
    {
        "name": "test monster 2",
        "type": "test",
        "level": 0,
        "health": 0,
    },
    {
        "name": "test monster 3",
        "type": "test",
        "level": 0,
        "health": 0,
    },
]).json()

True

## Find Monsters

In [10]:
requests.put(url, json={
    "query": {"type": "test"},
    "projection": {"name": True},
}).json()

[{'name': 'test monster 1'},
 {'name': 'test monster 2'},
 {'name': 'test monster 3'},
 {'name': 'test monster 1'},
 {'name': 'test monster 2'},
 {'name': 'test monster 3'},
 {'name': 'test monster 1'},
 {'name': 'test monster 2'},
 {'name': 'test monster 3'},
 {'name': 'test monster 1'},
 {'name': 'test monster 2'},
 {'name': 'test monster 3'}]

### Fail Case 1: This should be rejected by the API - field required 'health'

In [11]:
requests.post(url, json=[{
    "name": "test 002",
    "type": "test",
    "level": 0,
}]).json()

{'detail': [{'loc': ['body', 0, 'health'],
   'msg': 'field required',
   'type': 'value_error.missing'}]}

### Fail Case 2: This should be rejected by the API - extra fields not permitted

In [12]:
requests.post(url, json=[{
    "name": "test 003",
    "type": "test",
    "level": 0,
    "health": 0,
    "foreign": "field",
}]).json()

{'detail': [{'loc': ['body', 0, 'foreign'],
   'msg': 'extra fields not permitted',
   'type': 'value_error.extra'}]}

### Check How Many 'test' monsters exist

In [13]:
requests.put(url, json={
    "query": {"type": "test"},
    "projection": {},
}).json()

[{'name': 'test monster 1', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 2', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 3', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 1', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 2', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 3', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 1', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 2', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 3', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 1', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 2', 'type': 'test', 'level': 0, 'health': 0},
 {'name': 'test monster 3', 'type': 'test', 'level': 0, 'health': 0}]

### Update Many

In [14]:
requests.patch(url, json={
    "query": {"type": "test"},
    "update": {"level": 10},
}).json()

True

In [15]:
requests.put(url, json={
    "query": {"type": "test"},
    "projection": {"health": False, "type": False},
}).json()

[{'name': 'test monster 1', 'level': 10},
 {'name': 'test monster 2', 'level': 10},
 {'name': 'test monster 3', 'level': 10},
 {'name': 'test monster 1', 'level': 10},
 {'name': 'test monster 2', 'level': 10},
 {'name': 'test monster 3', 'level': 10},
 {'name': 'test monster 1', 'level': 10},
 {'name': 'test monster 2', 'level': 10},
 {'name': 'test monster 3', 'level': 10},
 {'name': 'test monster 1', 'level': 10},
 {'name': 'test monster 2', 'level': 10},
 {'name': 'test monster 3', 'level': 10}]

### Delete Many

In [16]:
requests.delete(url, json={
    "type": "test",
}).json()

True

Should be empty list

In [17]:
requests.put(url, json={
    "query": {"type": "test"},
    "projection": {"health": False, "type": False},
}).json()

[]

### Checksum

In [19]:
current_shape = DataFrame(requests.get(url).json()).shape
assert original_shape == current_shape

### Now For Something Sinister!

In [20]:
requests.patch(url, json={
    "query": {},
    "update": {"type": "Mushroom"},
}).json()

True

In [21]:
DataFrame(requests.get(url).json())

,name,type,level,health
0,Orc Captain,Mushroom,33,170
1,Mummy,Mushroom,38,209
2,Fire Mephit,Mushroom,33,169
3,Cave Troll,Mushroom,40,191
4,Black Dragon,Mushroom,75,436
...,...,...,...,...
995,Earth Mephit,Mushroom,32,179
996,Troll,Mushroom,39,206
997,Ghost,Mushroom,54,311
998,Black Dragon Welp,Mushroom,44,235
